In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import matplotlib.font_manager as fm
import scipy.linalg as la
import seaborn as sn

# --- plot aesthetical setup --- #

font_path = "cmunci.ttf"
custom_font = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = custom_font.get_name()
sn.set(font=custom_font.get_name())

base = {
    "axes.facecolor": "white",
    "axes.edgecolor": "black",
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.color": "black",
    "ytick.color": "black",
    "xtick.bottom": True,
    "xtick.top": True,
    "ytick.left": True,
    "ytick.right": True,
}

sn.set_theme(context='notebook', style=base, palette='rocket_r', color_codes=True, rc=None)

In [46]:
# --- import eigenvalues and eigenvectors calculated previously --- #

eigenvalues = np.loadtxt('Eigenvalues5.txt')
eigenvectors = np.loadtxt('Eigenvectors5.txt')

In [47]:
# --- global variables --- #

hbar = 0.658 # eV fs
m0 = 5.6856800 # fs2 eV/nm2
mu = 0.108 * m0 # fs2 eV/nm2
kb = 8.681e-5 # ev K {-1}

veps0 = 5.52638e-2 # c{2} eV{-1} mum{-1}
vepsS = 3.32 # dimless
vepsW = 6.10 # dimless

Lwell = 0.636 # nm
A = Lwell**2 # nm{2}
r0 = Lwell * vepsW/(2*vepsS) # nm

EgZero = 2.565 # ev - zero temperature bandgap energy
n = 1.82 # dimless - refractive index of the medium 
M = 0.44 * m0 # fs2 eV/nm2 - total exciton mass
Msigma = 8 # fs ev nm{-1} - optical matrix element for interband transition
GammaZero = 0.0067 # eV - defect-induced line widgth at T = 0K

DAC = 1.9 # eV - deformation potential for acoustic phonons
DOP = 158 # eV/nm - deformation potential for optical phonons

cAC = 2100 # m/s - acoustic phonon charactetistic velocity
c0 = 299.792 # nm/fs - speed of light in vacuum
hwOP = 35 # meV - optical phonon scattering

alphaOne = -1e-4 # material specific constant for Varshni shift
alphaTwo = 250 # material specific constant for Varshni shift

# --- simulation variables --- #

k_max = .75
N = 500
dk = k_max/N

k_vals = np.linspace(dk, k_max, N)
energies = np.linspace(-2, 3, 10000)

In [48]:
def BandGapEnergy(mu, T):
    return (EgZero - (alphaOne*T**2)/(alphaTwo + T)) - eigenvalues[mu]

def RadiativeRecom(mu, eigenvalues, eigenvectors): # Temperature independent radiactive recombination of eletrons
    wavefunction = FFT(mu, eigenvectors)
    C = (hbar**2 * Msigma**2) / (2 * m0**2 * veps0 * A * n * c0)

    return C * wavefunction / eigenvalues[mu]

def FFT(mu, eigenvectors): # inverse fourier transform of the amplitude of the wavefunctions
    C = A * dk / (2*np.pi)
    integral = np.sum(eigenvectors[mu][-500:] * k_vals)

    return (C * integral)**2

def Boltzmann(T, mu, eigenvalues):
    ZVec = np.sum(np.exp(eigenvalues/(kb*T)))
    BVec = np.exp(eigenvalues[mu]/(kb*T))/ZVec

    return BVec

In [49]:
def Photoluminescence(energy_range, T, eigenvalues, eigenvectors):

    PL = []
    C = 2 * Msigma**2 / hbar

    for E in energy_range:
        I = 0

        for mu in range(len(eigenvalues)):

            B = FFT(mu, eigenvectors) * RadiativeRecom(mu, eigenvalues, eigenvectors) * Boltzmann(T, mu, eigenvalues)
            denominator = (BandGapEnergy(mu, T) - E)**2 + (RadiativeRecom(mu, eigenvalues, eigenvectors) + GammaZero)**2
            I += B/denominator

        PL.append(C*I)

    return PL

In [50]:
def fwhm(x: np.ndarray, y: np.ndarray) -> float:
    
    half_max = np.max(y) / 2 
    indices = np.where(y >= half_max)[0]
    
    x1 = np.interp(half_max, [y[indices[0] - 1], y[indices[0]]], [x[indices[0] - 1], x[indices[0]]])
    x2 = np.interp(half_max, [y[indices[-1]], y[indices[-1] + 1]], [x[indices[-1]], x[indices[-1] + 1]])
    
    return x2 - x1

In [51]:
def DataAnalysis(temps):

    E = np.linspace(-2, 3, 10000)
    data = {"Energy": E}

    peaksE = []
    temperature = []

    # Run simulation
    for T in temps:
        PL = Photoluminescence(E, T, eigenvalues, eigenvectors)
        data[f'{T}'] = PL

        index = PL.index(max(PL))
        peaksE.append(E[index])
        temperature.append(T)

    df = pd.DataFrame(data)
    max_peak = np.max(df)

    fig = plt.figure(figsize=(7,9))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 2])

    # Plot evolution of the Lorentzian peak position
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    gs.update(hspace=0)

    ax1.xaxis.tick_top()
    ax2.xaxis.tick_bottom()
    ax1.yaxis.tick_left()
    ax2.yaxis.tick_left()
    ax1.xaxis.set_label_position('top')

    # Plot peak evoluton w.r.t temperature
    ax1.plot(temperature, peaksE, marker='X', linewidth=3, linestyle='--', color='darkblue')
    ax1.set_ylabel('Peak energy (eV)', fontsize=18)
    ax1.set_xlabel('Temperature (K)', fontsize=18)

    ax1.tick_params(axis="both", labelsize=18)
    ax1.tick_params(axis="both", which="major", direction="in", length=6, width=2)
    ax1.tick_params(axis="both", which="minor", direction="in", length=3, width=1.0)

    ax1.xaxis.set_major_locator(ticker.MultipleLocator(100))
    ax1.xaxis.set_minor_locator(ticker.MultipleLocator(50))
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(0.02))
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))

    ax1.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

    # Plot PL spectrum
    for column in df.columns[1:]:
        ax2.plot(df["Energy"], df[column]/max_peak, label=f'$T = {column} K$', linewidth=3)
    
        ax2.set_xlabel('Energy (eV)', fontsize=18)
        ax2.set_ylabel('Photoluminescence (normalised)', fontsize=18)
        ax2.set_xlim(2.2, 2.6)

        ax2.tick_params(axis="both", labelsize=18)
        ax2.tick_params(axis="both", which="major", direction="in", length=6, width=2)
        ax2.tick_params(axis="both", which="minor", direction="in", length=3, width=1.0)

        ax2.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
        ax2.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
        ax2.yaxis.set_major_locator(ticker.MultipleLocator(0.20))
        ax2.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))

        ax2.legend(fontsize=16)

    plt.savefig('PLPP.pdf', dpi=600)

    return df
    
        

In [52]:
def SecondaryPeak():

    PL5000 = Photoluminescence(np.linspace(-2, 10, 10000), 5000, eigenvalues, eigenvectors)
    PL10000 = Photoluminescence(np.linspace(-2, 10, 10000), 10000, eigenvalues, eigenvectors)

    maximum = np.max([np.max(PL5000), np.max(PL10000)])

    fig, ax = plt.subplots(2, 1, figsize=(7, 10), layout='constrained')

    highlight = patches.Rectangle((3.2, -0.01), 0.35, 0.21, linewidth=1, edgecolor='black', facecolor='none')
    
    ax[0].set_title('(a)', fontsize=16)
    ax[0].plot(np.linspace(-2, 10, 10000), PL5000/maximum, label='T = 5000K', lw=3)
    ax[0].plot(np.linspace(-2, 10, 10000), PL10000/maximum, label='T = 10000K', lw=3)
    ax[0].set_xlim(2.5, 3.75)
    ax[0].tick_params(labelsize=16)
    ax[0].set_ylabel('Photoluminescence (normalised)', fontsize=16)
    ax[0].add_patch(highlight)
    ax[0].text(3.6, 0.16, '(b)', color='black', fontsize=16, ha='center')
    ax[0].legend(fontsize=16)

    ax[0].xaxis.set_major_locator(ticker.MultipleLocator(0.25))
    ax[0].xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    ax[0].yaxis.set_major_locator(ticker.MultipleLocator(0.2))
    ax[0].yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

    ax[1].set_title('(b)', fontsize=16)
    ax[1].plot(np.linspace(-2, 10, 10000), PL5000/maximum, label='T = 5000K', lw=4)
    ax[1].plot(np.linspace(-2, 10, 10000), PL10000/maximum, label='T = 10000K', lw=4)
    ax[1].set_xlim(3.2, 3.55)
    ax[1].set_ylim(0, 0.2)
    ax[1].set_xlabel('Energy (eV)', fontsize=16)
    ax[1].tick_params(labelsize=16)
    ax[1].set_ylabel('Photoluminescence (normalised)', fontsize=16)

    ax[1].xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax[1].xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    ax[1].yaxis.set_major_locator(ticker.MultipleLocator(0.2))
    ax[1].yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

    plt.savefig('highTemperature.pdf', dpi=600)